In [1]:
import glob
import io
import os
import re
import requests
import zipfile
from xml.etree import ElementTree as ET
from bs4 import BeautifulSoup
from IPython.display import HTML

In [2]:
ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15'
url = 'https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.verb=W1E63021CXP002002DSPSch&uji.bean=ee.bean.parent.EECommonSearchBean&PID=W1E63021&TID=W1E63021&SESSIONKEY=1589338583099&lgKbn=2&pkbn=0&skbn=1&dskb=&askb=&dflg=0&iflg=0&preId=1&xbr=on&sec=&scc=4755&shb=&snm=&spf1=1&spf2=1&iec=&icc=&inm=&spf3=1&fdc=&fnm=&spf4=1&spf5=2&otd=120&cal=1&era=R&yer=&mon=&psr=1&pfs=5&row=100&idx=0&str=&kbn=1&flg=&syoruiKanriNo=&no=S100IAD1'
res = requests.get(url, headers={'user-agent': ua})

In [3]:
soup = BeautifulSoup(res.text)

In [4]:
# テーブルを取得
table = soup.select_one('table.resultTable.table_cellspacing_1.table_border_1.mb_6')

# trの一覧を取得
for tr in table.select('tr'):
    # 有価証券報告書なのか判断する
    a = tr.select_one('a')
    if a.string and re.search('\s+有価証券報告書', a.string):
        document_title = a.string.strip()
        img = tr.select_one('img[alt="XBRL"]')
        search = re.search('\((.*?)\)', img.parent['onclick'])
        params = list(map(lambda s: s.strip(), re.sub('\'', '', search[1]).split(',')))
        verb = params[0]
        appendix = params[4]
        xbrl_url = f'https://disclosure.edinet-fsa.go.jp/E01EW/download?1589344061616&uji.bean=ee.bean.parent.EECommonSearchBean&uji.verb={verb}&SESSIONKEY=1589344061363&s=S100IAD1&{appendix}'
        res = requests.get(xbrl_url)
        zip_obj = zipfile.ZipFile(io.BytesIO(res.content))
        for filepath in zip_obj.namelist():
            if 'PublicDoc' in filepath and '.xbrl' in filepath:
                filename = os.path.basename(filepath)
                print(filename)
                root = ET.fromstring(zip_obj.read(filepath))
                break
        break

jpcrp030000-asr-001_E05080-000_2019-12-31_01_2020-03-27.xbrl


In [5]:
for child in root:
    if child.text and re.search('財務諸表等', child.text):
        print(f'----- {child.tag} -----')
        print(child.text[:1000])
        print(f'----- {child.tag} -----', end='\n\n')

----- {http://disclosure.edinet-fsa.go.jp/jpcrp030000/asr/001/E05080-000/2019-12-31/01/2020-03-27}ManagementAnalysisOfFinancialPositionOperatingResultsAndCashFlowsTextBlock -----

<h3 class="smt_head2" style="font-family:&apos;ＭＳ ゴシック&apos;;">３ 【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】</h3>
<h4 class="smt_head3" style="line-height:15pt;">(1) 経営成績等の状況</h4><p class="smt_text3" style="orphans:0;widows:0;text-align:justify;padding-left:18pt;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;text-indent:9pt;font-family:&apos;ＭＳ 明朝&apos;;letter-spacing:0pt;line-height:15pt;">当社グループは、経営者が意思決定する際に使用する社内指標（以下、Non-GAAP指標）及びIFRSに基づく指標の双方によって、連結経営成績を開示しています。</p><p class="smt_text3" style="orphans:0;widows:0;text-align:justify;padding-left:18pt;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;text-indent:9pt;font-family:&apos;ＭＳ 明朝&apos;;letter-spacing:0pt;line-height:15pt;">Non-GAAP営業利益は、IFRSに基づく営業利益（以下、IFRS営業利益）から、当社グループが定める非経常的な項目やその他の調整項目を控除したものです。経営者は、Non-GAAP指標を開示することで、ステークホルダーにとって同業他社比較や過年度比較が容易に

In [6]:
balance_sheet_text_block = root.find('{http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2018-03-31/jpcrp_cor}BalanceSheetTextBlock')
HTML(balance_sheet_text_block.text)

In [7]:
soup = BeautifulSoup(balance_sheet_text_block.text)

In [10]:
tag_p = soup.find_all('p')

In [11]:
for tag in tag_p:
    print(tag.text)

 
 
 
 
 
 
 
 
 
 
 
(単位：百万円)
 
 
 
 
 
 
 
 
 
 
前事業年度
(2018年12月31日)
当事業年度
(2019年12月31日)
資産の部
 
 
 
流動資産
 
 
 
 
現金及び預金
※1 152,732
※1 180,209
 
 
売掛金
※1 121,955
※1 132,027
 
 
商品
7,204
9,448
 
 
貯蔵品
406
423
 
 
前払費用
13,484
15,211
 
 
未収入金
※1 141,980
※1 205,744
 
 
未収還付法人税等
1,018
1,958
 
 
関係会社短期貸付金
※2 10,007
※2 128,166
 
 
その他
※1 5,789
※1 13,157
 
 
貸倒引当金
△660
△352
 
 
流動資産合計
453,918
685,994
 
固定資産
 
 
 
 
有形固定資産
 
 
 
 
 
建物
6,968
8,602
 
 
 
機械装置及び運搬具
668
718
 
 
 
工具、器具及び備品
7,071
6,033
 
 
 
土地
－
478
 
 
 
建設仮勘定
514
4,456
 
 
 
その他
237
5,879
 
 
 
有形固定資産合計
15,460
26,169
 
 
無形固定資産
 
 
 
 
 
のれん
7,618
10,877
 
 
 
特許権
1,445
1,030
 
 
 
商標権
92
365
 
 
 
ソフトウエア
40,751
45,212
 
 
 
ソフトウエア仮勘定
7,806
10,097
 
 
 
その他
15,530
34,490
 
 
 
無形固定資産合計
73,245
102,073
 
 
投資その他の資産
 
 
 
 
 
投資有価証券
10,578
6,786
 
 
 
関係会社株式
1,180,982
1,111,883
 
 
 
関係会社出資金
645
645
 
 
 
関係会社長期貸付金
※2 9,626
※2 8,894
 
 
 
破産更生債権等
3,158
1,478
 
 
 
長期前払費用
4,471
2,450
 
 
 
敷金及び保証金
5,991
11,141
 
 
 
繰延税金資産
40,580
5